<h4>Притула М.І. 124М-22-1 Лаб№1 Варіант 4</h4>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import scipy.stats as stats
from scipy.stats import chisquare
from scipy.linalg import qr, svd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import r2_score

In [24]:
 
# # Reading the csv file
# df_new = pd.read_csv('lab1var4.csv')
 
# # saving xlsx file
# GFG = pd.ExcelWriter('lab1var4.xlsx')
# df_new.to_excel(GFG, index=False)
 
# GFG.save()

df2 = pd.read_excel("lab1var4.xlsx")
df = pd.read_csv("lab1var4.csv")
#df = df.drop('i')
df.head()

,,,,,,,,,,,i;x1;x2;x3;x4;x5;x6;x7;x8;x9;x10;Y
1;65,00874936;6481,21935;0,661074824;2,876553232;64,78272555;921,8181598;1174,816715;3,774410008;17,35257422;84,0520333;163,932347
2;70,2763401;3071,648209;0,762387143;5,048825909;134,7876707;1021,56288;1180,805518;3,946170785;19,22267478;102,5531587;194,5852805
3;77,22778768;2789,703091;0,452849727;5,98496992;126,0284951;999,2627079;1168,262414;5,435010721;19,27894735;99,96533225;209,9461152
4;80,1324658;4906,129908;0,233223761;5,455784561;182,3906956;933,4107048;1136,308313;5,072537727;18,062633;80,31387606;204,425209
5;85,5030071;2467,673775;0,289269985;3,590832865;192,5299095;995,5665493;1106,339439;5,518109872;19,23258178;81,1080889;199,7102978


In [186]:
df.corr().style.background_gradient(cmap='coolwarm', axis = 1)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,Y
x1,1.000000,0.714065,0.138614,0.016944,0.997502,-0.427388,0.437014,0.032722,-0.291284,-0.367971,0.348212
x2,0.714065,1.000000,-0.009551,0.070721,0.711013,-0.270369,0.420237,0.082401,-0.219284,-0.215248,0.301205
x3,0.138614,-0.009551,1.000000,0.363561,0.133695,0.012228,0.268439,0.316138,-0.067817,0.066139,0.201043
x4,0.016944,0.070721,0.363561,1.000000,0.019091,0.009576,0.602699,0.988841,-0.405739,0.167726,0.290915
x5,0.997502,0.711013,0.133695,0.019091,1.000000,-0.450697,0.452107,0.035147,-0.285159,-0.379933,0.338417
x6,-0.427388,-0.270369,0.012228,0.009576,-0.450697,1.000000,-0.284777,0.032874,0.129037,0.431611,0.106631
x7,0.437014,0.420237,0.268439,0.602699,0.452107,-0.284777,1.000000,0.580803,-0.545060,-0.079306,0.310668
x8,0.032722,0.082401,0.316138,0.988841,0.035147,0.032874,0.580803,1.000000,-0.416262,0.236382,0.368890
x9,-0.291284,-0.219284,-0.067817,-0.405739,-0.285159,0.129037,-0.545060,-0.416262,1.000000,-0.186438,-0.432312
x10,-0.367971,-0.215248,0.066139,0.167726,-0.379933,0.431611,-0.079306,0.236382,-0.186438,1.000000,0.735928


Можна побачити високу кореляцію між x1 і x5 та x4 і x8

In [187]:
scaled_X = MinMaxScaler().fit_transform(df.drop('Y', axis = 1))
df_scaled = pd.DataFrame(scaled_X, columns = df.drop('Y', axis = 1).columns)
df_scaled['Y'] = df['Y']
df_scaled.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,Y
0,0.000000000000,0.036315568871,0.337099655302,1.000000000000,0.000000000000,0.496226066008,0.855496630569,1.000000000000,0.006154260113,0.740639154791,206.374194900000
1,0.114447361771,0.365011173899,0.021752157508,0.192602169109,0.053631326696,1.000000000000,0.546230101298,0.271761800930,0.232888891806,0.921889783048,226.768040800000
2,0.147407991821,0.092148468366,1.000000000000,0.241295582916,0.096215045266,0.636769485786,0.437539680062,0.216130571230,0.938100948049,1.000000000000,246.061648400000
3,0.194808143437,0.430772894931,0.183000123332,0.243253099951,0.139627712833,0.000000000000,0.382167086478,0.250258226186,0.000000000000,0.955845700333,245.008849500000
4,0.173720352565,0.085826951375,0.527631983482,0.178449741430,0.162121191055,0.275907144457,0.214730958898,0.236985478004,0.791656963078,0.800237396573,221.953931600000


In [188]:
def farrar_glauber(df):
    # Separate dependent and independent variables
    X = df.iloc[:,:-1]
    y = df.iloc[:,-1]
    
    # Calculate residual errors for each independent variable
    residuals = []
    for i in range(X.shape[1]):
        # Fit linear regression model on independent variable i
        X_temp = X.drop(columns=X.columns[i])
        model = np.linalg.lstsq(X_temp, X.iloc[:,i], rcond=None)
        y_pred = np.dot(X_temp, model[0])
        
        # Calculate residual errors and append to list
        residuals.append(X.iloc[:,i] - y_pred)
    
    # Calculate correlation matrix of residuals
    corr_matrix = np.corrcoef(residuals)
    n = corr_matrix.shape[0]
    
    # Calculate T values and p-values
    T_values = np.zeros((n,n))
    p_values = np.zeros((n,n))
    for i in range(n):
        for j in range(i+1, n):
            r = corr_matrix[i,j]
            T_values[i,j] = r * np.sqrt((n-2)/(1-r**2))
            p_values[i,j] = 2*(1 - stats.t.cdf(abs(T_values[i,j]), n-2))
    
    # Create DataFrame to store T values and p-values
    col_names = X.columns
    t_values_df = pd.DataFrame(T_values, columns=col_names, index=col_names)
    p_values_df = pd.DataFrame(p_values, columns=col_names, index=col_names)
    
    # Return results as tuple
    return t_values_df, p_values_df

In [189]:
t_values, p_values = farrar_glauber(df_scaled)

In [190]:
pd.set_option('display.float_format', lambda x: '%.12f' % x)

In [191]:
# print("T-values:\n", t_values)
# print("\n")
# p_values.apply(lambda x: '{0:.10f}'.format(x))
alpha = 0.05
p_values
# print("P-values:\n", p_values)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10
x1,0.000000000000,0.781277315820,0.970509786845,0.499338258108,0.000000009589,0.237672243507,0.599100337516,0.509069925667,0.826003158231,0.424482723666
x2,0.000000000000,0.000000000000,0.483094608575,0.781263678587,0.938545708263,0.927658527557,0.679998223672,0.784622501644,0.764704087895,0.784584728098
x3,0.000000000000,0.000000000000,0.000000000000,0.130821624156,0.853935498108,0.829706403938,0.837712198530,0.157809558090,0.469610471157,0.283239941330
x4,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.396485481712,0.706076189863,0.239577721509,0.000000034240,0.665639726921,0.037766755253
x5,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.223304957759,0.489329407872,0.410315737404,0.816623877416,0.350792954162
x6,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.731891356611,0.693905129844,0.372378946461,0.581430663250
x7,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.352437793649,0.709920212780,0.283668092612
x8,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.722769559122,0.033613885059
x9,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.755732279398
x10,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000,0.000000000000


Після проведення тесту Фаррара-Глобера pvalues x1->x5, x4->x8 менше рівня значимості 0.05, тому ці признаки дійсно мультиколінеарні, та одні з нах варто видалити для ліпшої якості моделі

In [192]:
X = df_scaled.drop('Y', axis=1)
vif = pd.DataFrame()
vif["features"] = X.columns
vif["vif_Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif

,features,vif_Factor
0,x1,1008.693003560425
1,x2,8.440753329428
2,x3,5.247967050803
3,x4,334.082646274669
4,x5,996.112772096229
5,x6,4.108966703364
6,x7,17.122096195874
7,x8,344.883568865679
8,x9,2.224306248957
9,x10,6.729657346498


Це не входить в лабораторну, але я прочитав декілька статей і там часто говорилося про vif фактор (Variance inflation factor). Він показує як на кожну незалежну змінну впливають інші незалежні змінні. Чим більше vif тим більше вплив. Вважають, якщо vif > 10, то є мультиколінеарність.

In [193]:
X_scaled = df_scaled.drop(['x4', 'x5'], axis = 1)

In [194]:
X = X_scaled.drop('Y', axis=1)
vif = pd.DataFrame()
vif["features"] = X.columns
vif["vif_Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif

,features,vif_Factor
0,x1,10.159943071377
1,x2,8.321793476903
2,x3,3.831216613102
3,x6,3.382048014504
4,x7,13.960324414448
5,x8,6.463805763475
6,x9,2.170022887644
7,x10,3.685750871173


Після видалення двох факторів видно як зменшився vif

In [195]:
X_scaled['Y'] = df['Y']

In [196]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Add-Del алгоритм

In [197]:
def add_delete_algorithm(train_data, test_data, max_factors):
    """
    Add-Delete algorithm for selecting informative features.

    Args:
        train_data (pd.DataFrame): Training data with features and target.
        test_data (pd.DataFrame): Test data with features and target.
        max_factors (int): Maximum number of features to consider.

    Returns:
        (list): List of selected features.
    """
    # Start with an empty set of features.
    features = set()

    # Step 2: Identify dependencies on training data and compute errors on test data.
    best_error = float('inf')
    best_feature = None
    for feature in train_data.drop('Y', axis=1).columns:
        # Add feature to set.
        features.add(feature)

        # Train model on training data.
        model = LinearRegression().fit(train_data[list(features)], train_data['Y'])

        # Evaluate model on test data.
        test_predictions = model.predict(test_data[list(features)])
        test_error = mean_squared_error(test_data['Y'], test_predictions)

        # Update best feature if necessary.
        if test_error < best_error:
            best_error = test_error
            best_feature = feature

        # Remove feature from set.
        features.remove(feature)

    # Step 3: Add best feature to set.
    features.add(best_feature)

    # Step 4: Repeat until maximum number of features is reached.
    while len(features) < max_factors:
        # Step 5: Identify dependencies on training data and compute errors on test data.
        best_error = float('inf')
        best_feature = None
        for feature in train_data.drop('Y', axis=1).columns:
            if feature in features:
                continue

            # Add feature to set.
            features.add(feature)

            # Train model on training data.
            model = LinearRegression().fit(train_data[list(features)], train_data['Y'])

            # Evaluate model on test data.
            test_predictions = model.predict(test_data[list(features)])
            test_error = mean_squared_error(test_data['Y'], test_predictions)

            # Update best feature if necessary.
            if test_error < best_error:
                best_error = test_error
                best_feature = feature

            # Remove feature from set.
            features.remove(feature)

        # Step 6: Add best feature to set.
        features.add(best_feature)

    # Step 7: Return list of selected features.
    return list(features)

In [198]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [199]:
# Split data into training and test sets.
train_data, test_data = train_test_split(X_scaled, test_size=0.2, random_state=42)

# Run Add-Delete algorithm.
selected_features = add_delete_algorithm(train_data, test_data, max_factors=5)

# Train final model on selected features.
final_model = LinearRegression().fit(train_data[selected_features], train_data['Y'])

# Evaluate final model on test data.
test_predictions = final_model.predict(test_data[selected_features])
test_error = mean_squared_error(test_data['Y'], test_predictions)
print(f'Test error: {test_error:.2f}')

Test error: 7.36


In [200]:
selected_features

['x1', 'x10', 'x8', 'x7', 'x6']

Del-Add алгоритм

In [201]:
def del_add_algorithm(x_train, y_train, x_test, y_test, max_factors):
    n_features = x_train.shape[1]
    selected_factors = set(range(n_features))
    curr_factors = selected_factors.copy()
    curr_score = np.inf
    
    while len(selected_factors) <= max_factors:
        # Try deleting each factor from the current set
        for factor in curr_factors:
            new_factors = curr_factors.copy()
            new_factors.remove(factor)
            model = LinearRegression().fit(x_train[:, list(new_factors)], y_train)
            score = mean_squared_error(y_test, model.predict(x_test[:, list(new_factors)]))
            
            # If the score improved, update the current set of factors and score
            if score < curr_score:
                curr_factors = new_factors
                curr_score = score
        
        # Try adding each factor that isn't in the current set
        for factor in selected_factors - curr_factors:
            new_factors = curr_factors.copy()
            new_factors.add(factor)
            model = LinearRegression().fit(x_train[:, list(new_factors)], y_train)
            score = mean_squared_error(y_test, model.predict(x_test[:, list(new_factors)]))
            
            # If the score improved, update the current set of factors and score
            if score < curr_score:
                curr_factors = new_factors
                curr_score = score
        
        # If no improvements were made, stop the algorithm
        if curr_factors == selected_factors:
            break
        
        # Otherwise, update the selected set of factors and continue
        selected_factors = curr_factors.copy()
    
    return selected_factors

In [202]:
X_train = train_data.drop('Y', axis = 1)
y_train = train_data['Y']
X_test = test_data.drop('Y', axis = 1)
y_test = test_data['Y']
selected_features = del_add_algorithm(X_train, y_train, X_test, y_test, 5)

In [203]:
list(selected_features)

[0, 1, 2, 3, 4, 5, 6, 7]

In [205]:
# Train final model on selected features.
final_model = LinearRegression().fit(train_data[X_train.columns[list(selected_features)]], train_data['Y'])

# Evaluate final model on test data.
test_predictions = final_model.predict(test_data.drop('Y', axis = 1))
test_error = mean_squared_error(test_data['Y'], test_predictions)
print(f'Test error: {test_error:.2f}')

Test error: 12.42


**Висновок**:<br>
Тест Фаррара-Глобера показав колінеарність між двома незалежними змінними.<br>
Add-Del алгоритм повернув не всі фічі.<br> Del-Add алгоритм повернув всі фічі.<br>
Add-Del відпрацював трішки ліпше на тестовій вибірці, але взагалом обидва алгоритми відпрацювали добре.